In [1]:
import torch
from torch import nn

In [6]:
cfgs = { "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
         "E": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"] }

In [33]:
class VGG(nn.Module):
    def __init__(self, cfg, batch_norm = False, num_classes = 1000, init_weights = True, drop_p = 0.5):
        super().__init__()

        # Conv part
        self.features = self.make_layers(cfg, batch_norm)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7)) # 7x7 이 되도록 avg pooling 하는 녀석
                                                    # => 224 224가 아닌 녀석들도 마지막에는 7 7로 만들어서 MLP 통과 가능하게!

        # MLP part
        self.classifier = nn.Sequential(nn.Linear(512 * 7 * 7, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, num_classes))

        # Weight 초기화 -> 약간 국룰 국밥임
        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d): # Conv 이면 ~~~ -> m(객체)의 클래스가 nn.Conv2d인지 확인
                    nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")

                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0) # bias가 있으면 0으로 초기화 -> nn.init.constant_(tensor, value) -> m.bias tensor을 0으로

                elif isinstance(m, nn.Linear): # Linear(MLP)면 ~~~
                    nn.init.normal_(m.weight, 0, 0.01)
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def make_layers(self, cfg, batch_norm = False):
        layers = []
        in_channels = 3 # RGB니깐 통과되는 데이터가 지금 3 채널이니깐 이놈으로 스타트하고 점점 바뀜

        for v in cfg: # cfg = [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"]
            if type(v) == int:
                if batch_norm:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1, bias=False), # 어차피 BN에 bias 포함
                               nn.BatchNorm2d(v),
                               nn.ReLU()]
                else:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1),
                               nn.ReLU()]
                in_channels = v # in_channel <-> v 관계!!!!
            else:
                layers += [nn.MaxPool2d(2)]

        return nn.Sequential(*layers)

In [3]:
avgpool = nn.AdaptiveAvgPool2d((4, 4))
print(avgpool(torch.randn(2,3,32,32)).shape)
x = torch.randn(2,3,2,2)
print(x)
print(avgpool(x)) # 작은 놈이 들어오면 늘려서라도 맞춰준다 # 값을 복제 시켜놓음

torch.Size([2, 3, 4, 4])
tensor([[[[-0.5900, -1.4273],
          [ 2.3369, -1.4783]],

         [[ 0.2351,  0.1589],
          [ 0.6665, -1.7890]],

         [[-1.4434, -0.4754],
          [-0.2010, -1.7372]]],


        [[[ 0.1398, -1.2128],
          [-0.6390,  0.4752]],

         [[ 0.2769, -1.6660],
          [-0.4427,  3.0673]],

         [[-0.9704,  0.2281],
          [ 0.0592,  0.6110]]]])
tensor([[[[-0.5900, -0.5900, -1.4273, -1.4273],
          [-0.5900, -0.5900, -1.4273, -1.4273],
          [ 2.3369,  2.3369, -1.4783, -1.4783],
          [ 2.3369,  2.3369, -1.4783, -1.4783]],

         [[ 0.2351,  0.2351,  0.1589,  0.1589],
          [ 0.2351,  0.2351,  0.1589,  0.1589],
          [ 0.6665,  0.6665, -1.7890, -1.7890],
          [ 0.6665,  0.6665, -1.7890, -1.7890]],

         [[-1.4434, -1.4434, -0.4754, -0.4754],
          [-1.4434, -1.4434, -0.4754, -0.4754],
          [-0.2010, -0.2010, -1.7372, -1.7372],
          [-0.2010, -0.2010, -1.7372, -1.7372]]],


        [[[ 0.13

In [2]:
model = nn.Sequential(nn.Conv2d(3,32,3),
                      nn.ReLU(),
                      nn.Sequential(nn.Conv2d(32,64,3),
                                    nn.ReLU(),
                                    nn.Linear(64*8*8,30),
                                    nn.ReLU()),
                      nn.Linear(30,10))
[m for m in model.modules()]

[Sequential(
   (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
   (1): ReLU()
   (2): Sequential(
     (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
     (1): ReLU()
     (2): Linear(in_features=4096, out_features=30, bias=True)
     (3): ReLU()
   )
   (3): Linear(in_features=30, out_features=10, bias=True)
 ),
 Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1)),
 ReLU(),
 Sequential(
   (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
   (1): ReLU()
   (2): Linear(in_features=4096, out_features=30, bias=True)
   (3): ReLU()
 ),
 Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1)),
 ReLU(),
 Linear(in_features=4096, out_features=30, bias=True),
 ReLU(),
 Linear(in_features=30, out_features=10, bias=True)]

In [6]:
# model1 = nn.Sequential([nn.Linear(1,1),
#                        nn.Linear(1,1)]) # 리스트를 넣으면 안돼요!

model2 = nn.Sequential(nn.Linear(1,1),
                       nn.Linear(1,1))

# print(*[1,2])
# print([1,2])

# model3 = nn.Sequential(*[nn.Linear(1,1),
#                          nn.Linear(1,1)])

In [34]:
model = VGG(cfgs["E"], batch_norm=True)
# print(model)

from torchinfo import summary
summary(model, input_size=(2,3,32, 32), device='cpu') # AdaptiveAvgPool 덕분에 224 224가 아닌 32 32도 512 7 7로 나온다..!

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [2, 1000]                 --
├─Sequential: 1-1                        [2, 512, 1, 1]            --
│    └─Conv2d: 2-1                       [2, 64, 32, 32]           1,728
│    └─BatchNorm2d: 2-2                  [2, 64, 32, 32]           128
│    └─ReLU: 2-3                         [2, 64, 32, 32]           --
│    └─Conv2d: 2-4                       [2, 64, 32, 32]           36,864
│    └─BatchNorm2d: 2-5                  [2, 64, 32, 32]           128
│    └─ReLU: 2-6                         [2, 64, 32, 32]           --
│    └─MaxPool2d: 2-7                    [2, 64, 16, 16]           --
│    └─Conv2d: 2-8                       [2, 128, 16, 16]          73,728
│    └─BatchNorm2d: 2-9                  [2, 128, 16, 16]          256
│    └─ReLU: 2-10                        [2, 128, 16, 16]          --
│    └─Conv2d: 2-11                      [2, 128, 16, 16]          147,

In [9]:
x = torch.randn(2,3,224,224)
print(model(x).shape)

torch.Size([2, 1000])


In [10]:
x = torch.randn(2,3,300,300)
print(model(x).shape)

torch.Size([2, 1000])


In [11]:
x = torch.randn(2,3,32,32)
print(model(x).shape)

torch.Size([2, 1000])


In [11]:
# nn.MaxPool2d(2)(torch.randn(2,3,1,1)) # error!

In [13]:
from multiclass_function_2 import *
import torch
from torchvision import datasets, transforms
import copy

In [28]:
BATCH_SIZE = 64
LR = 2e-3
LR_STEP = 3 # for LR STEP
LR_GAMMA = 0.9 # for LR STEP
EPOCH = 30
TRAIN_RATIO = 0.8
criterion = nn.CrossEntropyLoss()
new_model_train = True
model_type = "VGG_Test"
dataset = "CIFAR10"
save_model_path = f"./result/{model_type}_{dataset}.pt"
save_history_path = f"./result/{model_type}_history_{dataset}.pt"

transform_train = transforms.Compose([
    transforms.ToTensor()])
transform_test = transforms.ToTensor()

In [15]:
train_DS = datasets.CIFAR10(root = '/Users/sanghyun/Desktop/GIT_Folder', train = True, download=False, transform=transform_train)
test_DS = datasets.CIFAR10(root  = '/Users/sanghyun/Desktop/GIT_Folder', train = False, download=False, transform=transform_test)

In [17]:
NoT = int(len(train_DS)*TRAIN_RATIO)
NoV = len(train_DS) - NoT

train_DS, val_DS= torch.utils.data.random_split(train_DS, [NoT, NoV])

val_DS.transform = transform_test # test의 transform 적용!

train_DL = torch.utils.data.DataLoader(train_DS, batch_size=BATCH_SIZE, shuffle=True)
val_DL = torch.utils.data.DataLoader(val_DS, batch_size=BATCH_SIZE, shuffle=True) # 이미지 변형 체크 할땐 False로
test_DL = torch.utils.data.DataLoader(test_DS, batch_size=BATCH_SIZE, shuffle=True) # 이미지 변형 체크 할땐 False로

In [36]:
class VGG_Test(nn.Module):
    def __init__(self, cfg, batch_norm = False, num_classes = 1000, init_weights = True, drop_p = 0.5):
        super().__init__()

        # Conv part
        self.features = self.make_layers(cfg, batch_norm)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7)) # 7x7 이 되도록 avg pooling 하는 녀석
                                                    # => 224 224가 아닌 녀석들도 마지막에는 7 7로 만들어서 MLP 통과 가능하게!

        # MLP part
        self.classifier = nn.Sequential(nn.Linear(512 * 7 * 7, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, num_classes))

        # Weight 초기화 -> 약간 국룰 국밥임
        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d): # Conv 이면 ~~~ -> m(객체)의 클래스가 nn.Conv2d인지 확인
                    nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")

                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0) # bias가 있으면 0으로 초기화 -> nn.init.constant_(tensor, value) -> m.bias tensor을 0으로

                elif isinstance(m, nn.Linear): # Linear(MLP)면 ~~~
                    nn.init.normal_(m.weight, 0, 0.01)
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        if x.device.type == "mps":
            x = x.cpu()
            x = self.avgpool(x)
            x = x.to("mps")
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def make_layers(self, cfg, batch_norm = False):
        layers = []
        in_channels = 3 # RGB니깐 통과되는 데이터가 지금 3 채널이니깐 이놈으로 스타트하고 점점 바뀜

        for v in cfg: # cfg = [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"]
            if type(v) == int:
                if batch_norm:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1, bias=False), # 어차피 BN에 bias 포함
                               nn.BatchNorm2d(v),
                               nn.ReLU()]
                else:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1),
                               nn.ReLU()]
                in_channels = v # in_channel <-> v 관계!!!!
            else:
                layers += [nn.MaxPool2d(2)]

        return nn.Sequential(*layers)

In [37]:
model = VGG_Test(cfgs["A"], batch_norm=True,num_classes=10, init_weights=True, drop_p=0.5)
# print(model)

summary(model, input_size=(2,3,32, 32), device='mps')

Layer (type:depth-idx)                   Output Shape              Param #
VGG_Test                                 [2, 10]                   --
├─Sequential: 1-1                        [2, 512, 1, 1]            --
│    └─Conv2d: 2-1                       [2, 64, 32, 32]           1,728
│    └─BatchNorm2d: 2-2                  [2, 64, 32, 32]           128
│    └─ReLU: 2-3                         [2, 64, 32, 32]           --
│    └─MaxPool2d: 2-4                    [2, 64, 16, 16]           --
│    └─Conv2d: 2-5                       [2, 128, 16, 16]          73,728
│    └─BatchNorm2d: 2-6                  [2, 128, 16, 16]          256
│    └─ReLU: 2-7                         [2, 128, 16, 16]          --
│    └─MaxPool2d: 2-8                    [2, 128, 8, 8]            --
│    └─Conv2d: 2-9                       [2, 256, 8, 8]            294,912
│    └─BatchNorm2d: 2-10                 [2, 256, 8, 8]            512
│    └─ReLU: 2-11                        [2, 256, 8, 8]            --


In [29]:
model = VGG("A", batch_norm=True, num_classes=10, init_weights=True, drop_p=0.5).to(DEVICE)

In [38]:
if new_model_train:
    optimizer = optim.Adam(model.parameters(), lr=LR)
    # loss_history = Train(model, train_DL, criterion, optimizer, EPOCH, )
    loss_history = Train(model, train_DL, val_DL, criterion, optimizer, EPOCH,
          BATCH_SIZE, TRAIN_RATIO,
          save_model_path, save_history_path)

    torch.save(model, save_model_path)

Epoch: 1, current_LR = 0.002


train loss: 3.72873, val loss: 2.30292 
train acc: 9.7 %, val acc: 10.0 %, time: 46 s
--------------------
Epoch: 2, current_LR = 0.002


train loss: 2.30293, val loss: 2.30306 
train acc: 9.8 %, val acc: 9.4 %, time: 40 s
--------------------
Epoch: 3, current_LR = 0.002


train loss: 2.30293, val loss: 2.30294 
train acc: 9.9 %, val acc: 10.0 %, time: 40 s
--------------------
Epoch: 4, current_LR = 0.002


train loss: 2.30276, val loss: 2.30355 
train acc: 9.9 %, val acc: 9.8 %, time: 40 s
--------------------
Epoch: 5, current_LR = 0.002


KeyboardInterrupt: 